# EDA 및 baseline 코드 확인

In [1]:
import pandas as pd
import os
import torch
import easydict
import numpy as np
from sklearn.preprocessing import LabelEncoder
import time
import datetime
from datetime import datetime
import random
import wandb

In [2]:
df = pd.read_csv('/opt/ml/p4-dkt-freshtomato/input/data/train_dataset/train_data.csv') 

In [3]:
df # assessmentItemID, testId, KnowledgeTag를 

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836


In [4]:
df = df.sort_values(by=['userID','Timestamp'], axis=0) # 차이가 없어보이는데 이거는 왜 하는거지?

In [5]:
df

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225
...,...,...,...,...,...,...
2266581,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438
2266582,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836
2266583,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836
2266584,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836


## USER의 TestID(시험지)별 정답률 추가

In [6]:
type(df['answerCode'][0])

numpy.int64

In [7]:
total_count, answer_count = 1, 0
pr_user_id, pr_test_id, pr_answer_code = 0, 'A060000001', 1
user_test_correct_ratio = []

for user_id, test_id, answer_code in zip(df['userID'], df['testId'], df['answerCode']):
    if (user_id != pr_user_id) or (test_id != pr_test_id): # 다른 user 시작 or 다른 시험지 시작
        total_count, answer_count = 1, 0
        pr_user_id = user_id
        pr_test_id = test_id
    if answer_code == 1:
        answer_count += 1
    user_test_correct_ratio.append(answer_count/total_count)
    total_count += 1

In [8]:
user_test_correct_ratio = pd.DataFrame(user_test_correct_ratio, columns=['correctRatio'])

In [9]:
user_test_correct_ratio

,correctRatio
0,1.0
1,1.0
2,1.0
3,1.0
4,1.0
...,...
2266581,0.2
2266582,1.0
2266583,1.0
2266584,1.0


In [10]:
df = df.join(user_test_correct_ratio)

In [12]:
df[:15]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,correctRatio
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1.000000
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1.000000
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1.000000
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1.000000
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1.000000
5,0,A060001007,A060000001,1,2020-03-24 00:17:47,7225,1.000000
6,0,A060003001,A060000003,0,2020-03-26 05:52:03,7226,0.000000
7,0,A060003002,A060000003,1,2020-03-26 05:52:10,7226,0.500000
8,0,A060003003,A060000003,1,2020-03-26 05:53:14,7226,0.666667
9,0,A060003004,A060000003,1,2020-03-26 05:53:29,7226,0.750000


In [17]:
cr = np.load("/opt/ml/p4-dkt-freshtomato/code/asset/correctRate_classes.npy")
print(cr)
print(len(cr))

['0.0' '0.05555555555555555' '0.058823529411764705' '0.0625'
 '0.07692307692307693' '0.08333333333333333' '0.09090909090909091' '0.1'
 '0.1111111111111111' '0.125' '0.13333333333333333' '0.14285714285714285'
 '0.15384615384615385' '0.16666666666666666' '0.17647058823529413'
 '0.18181818181818182' '0.1875' '0.2' '0.21428571428571427'
 '0.2222222222222222' '0.23076923076923078' '0.25' '0.26666666666666666'
 '0.2727272727272727' '0.2857142857142857' '0.29411764705882354' '0.3'
 '0.3076923076923077' '0.3125' '0.3333333333333333' '0.35294117647058826'
 '0.35714285714285715' '0.36363636363636365' '0.3684210526315789' '0.375'
 '0.38095238095238093' '0.38461538461538464' '0.4' '0.4090909090909091'
 '0.4117647058823529' '0.4166666666666667' '0.42105263157894735'
 '0.42857142857142855' '0.43478260869565216' '0.4375' '0.4444444444444444'
 '0.45454545454545453' '0.4583333333333333' '0.46153846153846156'
 '0.4666666666666667' '0.47058823529411764' '0.4782608695652174' '0.5'
 '0.5217391304347826' '0

In [6]:
cate_cols = ['assessmentItemID', 'testId', 'KnowledgeTag']

for col in cate_cols:
    le = LabelEncoder() # LabelEncoder: 문자를 0부터 시작하는 정수형 숫자로 바꿔주는 기능을 제공
    #For UNKNOWN class
    a = df[col].unique().tolist() + ['unknown'] # unknown class를 추가하고, 정수형 숫자로 label을 변경
    le.fit(a) # 결과값을 asset 폴더 밑 ~.npy로 저장

    #모든 컬럼이 범주형이라고 가정
    df[col] = df[col].astype(str)
    test = le.transform(df[col])
    df[col] = test

def convert_time(s):
    timestamp = time.mktime(datetime.strptime(s, '%Y-%m-%d %H:%M:%S').timetuple()) # 날짜를 초 단위로 변경
    return int(timestamp)

df['Timestamp'] = df['Timestamp'].apply(convert_time) # -- 여기까지 결과가 get_train_data를 해서 가져오는 것!

In [7]:
columns = ['userID', 'assessmentItemID', 'testId', 'answerCode', 'KnowledgeTag']
group = df[columns].groupby('userID').apply(
        lambda r: (
            r['testId'].values, 
            r['assessmentItemID'].values,
            r['KnowledgeTag'].values,
            r['answerCode'].values
        )
    )
# userID 별로 다음의 value들을 묶어서 정보를 가지게 됨
'''
group -> len(group.values): 6,698 - train/test셋이 userID를 기준으로 90/10의 비율로 나누어져 있음 (총 7,442명)
    - user 0 -> len(group.values[0]): 4
        - testId
        - assessmentItemID
        - KnowledgeTag
        - answerCode
    - user 1
    ...
    - user 7441
'''

'\ngroup -> len(group.values): 6,698 - train/test셋이 userID를 기준으로 90/10의 비율로 나누어져 있음 (총 7,442명)\n    - user 0 -> len(group.values[0]): 4\n        - testId\n        - assessmentItemID\n        - KnowledgeTag\n        - answerCode\n    - user 1\n    ...\n    - user 7441\n'

In [13]:
data = group.values

In [14]:
random.seed(0) # fix to default seed 0
random.shuffle(data)

ratio = 0.1
size = int(len(data) * ratio)
data_1 = data[:size] # valid - 669
data_2 = data[size:] # train - 6029

In [19]:
from sklearn.model_selection import train_test_split
sk_data2, sk_data1 = train_test_split(data, test_size=0.1, random_state=0) # 6028, 670

### Lable에 사용되는 class 확인

In [20]:
itemID_class = np.load('asset/assessmentItemID_classes.npy')

In [23]:
print(itemID_class)
print(len(itemID_class))

['A010001001' 'A010001002' 'A010001003' ... 'A090074005' 'A090074006'
 'unknown']
9455


In [27]:
knowledgeTag_class = np.load('asset/KnowledgeTag_classes.npy')

In [28]:
print(knowledgeTag_class)
print(len(knowledgeTag_class))

['1000' '1002' '1009' '1010' '1011' '10168' '10169' '10174' '10180'
 '10181' '10182' '10190' '10196' '10313' '10328' '10329' '10330' '10332'
 '10333' '10335' '10395' '10396' '10397' '10398' '10399' '10400' '10402'
 '10403' '10404' '10405' '10421' '10450' '10568' '10569' '10572' '10573'
 '10590' '10603' '10604' '10605' '10606' '10607' '10608' '10610' '10615'
 '10618' '10624' '10625' '10741' '10747' '10749' '10759' '10760' '10764'
 '10766' '10778' '10784' '10795' '10819' '10834' '10837' '10838' '10841'
 '1107' '1108' '1110' '1117' '1120' '11214' '11235' '11236' '11241'
 '11246' '11253' '11265' '11269' '11270' '11271' '116' '117' '1180' '1181'
 '1182' '1183' '1184' '1185' '1200' '1201' '1202' '1203' '121' '1248'
 '1249' '1250' '1251' '1252' '1253' '1255' '1256' '1257' '1259' '1260'
 '1261' '1262' '1263' '1264' '1266' '1267' '1268' '1269' '1271' '1273'
 '1356' '1394' '1395' '1396' '1397' '1422' '1439' '1442' '1443' '1444'
 '1450' '1451' '1452' '1453' '1454' '1455' '1456' '1458' '1463' '146

In [30]:
testId_class = np.load('asset/testId_classes.npy')

In [31]:
print(testId_class)
print(len(testId_class))

['A010000001' 'A010000002' 'A010000003' ... 'A090000073' 'A090000074'
 'unknown']
1538


In [3]:
import os
from datetime import datetime
import time
import tqdm
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder
import numpy as np
import torch

class Preprocess:
    def __init__(self,args):
        self.args = args
        self.train_data = None
        self.test_data = None
        

    def get_train_data(self):
        return self.train_data

    def get_test_data(self):
        return self.test_data

    def split_data(self, data, ratio=0.7, shuffle=True, seed=0):
        """
        split data into two parts with a given ratio.
        """
        if shuffle:
            random.seed(seed) # fix to default seed 0
            random.shuffle(data)

        size = int(len(data) * ratio) # data가 userID별로 묶여 있으므로, 이를 기준으로 나눔
        data_1 = data[:size]
        data_2 = data[size:]

        return data_1, data_2

    def __save_labels(self, encoder, name):
        le_path = os.path.join(self.args.asset_dir, name + '_classes.npy')
        np.save(le_path, encoder.classes_)

    def __preprocessing(self, df, is_train = True):
        cate_cols = ['assessmentItemID', 'testId', 'KnowledgeTag']

        if not os.path.exists(self.args.asset_dir):
            os.makedirs(self.args.asset_dir)
            
        for col in cate_cols:
            
            
            le = LabelEncoder() # LabelEncoder: 문자를 0부터 시작하는 정수형 숫자로 바꿔주는 기능을 제공
            if is_train:
                #For UNKNOWN class
                a = df[col].unique().tolist() + ['unknown']
                le.fit(a)
                self.__save_labels(le, col)
            else:
                label_path = os.path.join(self.args.asset_dir,col+'_classes.npy')
                le.classes_ = np.load(label_path)
                
                df[col] = df[col].apply(lambda x: x if x in le.classes_ else 'unknown')

            #모든 컬럼이 범주형이라고 가정
            df[col]= df[col].astype(str)
            test = le.transform(df[col])
            df[col] = test
            

        def convert_time(s):
            timestamp = time.mktime(datetime.strptime(s, '%Y-%m-%d %H:%M:%S').timetuple())
            return int(timestamp)

        df['Timestamp'] = df['Timestamp'].apply(convert_time)
        
        return df

    def __feature_engineering(self, df):
        #TODO
        return df

    def load_data_from_file(self, file_name, is_train=True):
        csv_file_path = os.path.join(self.args.data_dir, file_name)
        df = pd.read_csv(csv_file_path)#, nrows=100000)
        df = self.__feature_engineering(df)
        df = self.__preprocessing(df, is_train)

        # 추후 feature를 embedding할 시에 embedding_layer의 input 크기를 결정할때 사용

                
        self.args.n_questions = len(np.load(os.path.join(self.args.asset_dir,'assessmentItemID_classes.npy')))
        self.args.n_test = len(np.load(os.path.join(self.args.asset_dir,'testId_classes.npy')))
        self.args.n_tag = len(np.load(os.path.join(self.args.asset_dir,'KnowledgeTag_classes.npy')))
        

        df = df.sort_values(by=['userID','Timestamp'], axis=0)
        columns = ['userID', 'assessmentItemID', 'testId', 'answerCode', 'KnowledgeTag']
        group = df[columns].groupby('userID').apply(
                lambda r: (
                    r['testId'].values, 
                    r['assessmentItemID'].values,
                    r['KnowledgeTag'].values,
                    r['answerCode'].values
                )
            )

        return group.values

    def load_train_data(self, file_name):
        self.train_data = self.load_data_from_file(file_name)

    def load_test_data(self, file_name):
        self.test_data = self.load_data_from_file(file_name, is_train= False)




In [10]:
set_seed(args.seed)

preprocess = Preprocess(args)

In [11]:
preprocess.load_train_data(file_name)

train_data = preprocess.get_train_data()

In [12]:
type(train_data)

numpy.ndarray